# model 3

In [38]:
from abc import ABC
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from preprocessing import EmbeddingDataset
from model2_nn import train_and_plot
from torch.nn.utils import rnn

In [2]:
ds = EmbeddingDataset()
train_dataloader, test_dataloader = ds.get_data_loaders(batch_size=64)

prepering glove...


In [39]:
dev_dataloader = test_dataloader

In [42]:
class LSTM_NER_NN(nn.Module):
    def __init__(self, input_size=200, hidden_dim=50, num_classes=2, model_save_path=""):
        super().__init__()
        self.embedding_dim = input_size
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=self.hidden_dim, batch_first=True, bidirectional=True)
        self.hidden2tag = nn.Sequential(nn.ReLU(),
                                        nn.Linear(self.hidden_dim, num_classes))
        self.loss_fn = nn.NLLLoss()
        self.model_save_path = model_save_path
        self.num_classes = num_classes

    def forward(self, X_batch, y_batch=None):
        # X_batch is a list of tensors: size 200*sentence_size for each sentence (sentence_size differ between sentences)
        X_packed = rnn.pack_sequence(X_batch)
        lstm_out, _ = self.lstm(X_packed)
        tag_space = self.hidden2tag(lstm_out)
        tag_score = F.softmax(tag_space, dim=1)
        if y_batch is None:
            return tag_score, None
        loss = self.loss_fn(tag_score, y_batch)
        return tag_score, loss

In [43]:
def main():
    # option 1:
    # is identity - yes / no
    num_classes = 2
    # option 2:
    # different classification for different identities
    # num_classes = ?
    batch_size=64

    num_epochs = 5
    hidden_dim = 64
    # single vector size
    input_size = ds.VEC_DIM
    lr = 0.001

    model_save_path = f"LSTM_model_stateDict_batchSize_{batch_size}_hidden_{hidden_dim}_lr_{lr}.pt"
    LSTM_model = LSTM_NER_NN(
        input_size=input_size,
        num_classes=num_classes,
        hidden_dim=hidden_dim,
        model_save_path=model_save_path,
    )

    train_and_plot(
        NN_model=LSTM_model,
        train_loader=train_dataloader,
        num_epochs=num_epochs,
        batch_size=batch_size,
        val_loader=dev_dataloader,
        lr=lr
    )

In [44]:
main()

No GPU available, training on CPU.


  0%|          | 0/981 [00:00<?, ?it/s]


RuntimeError: input must have 2 dimensions, got 1

In [41]:
a = torch.Tensor([[1], [2], [3]]) 
b = torch.Tensor([[4], [5]]) 
c = torch.Tensor([[6]]) 
seq = rnn.pack_sequence([a, b, c]) 
lstm = torch.nn.LSTM(input_size=1, hidden_size=3) 
lstm(seq)

(PackedSequence(data=tensor([[ 0.0171,  0.2328, -0.0830],
         [-0.0291,  0.1918, -0.0890],
         [-0.0135,  0.1229, -0.0555],
         [-0.0063,  0.3442, -0.1289],
         [-0.0256,  0.2454, -0.0948],
         [-0.0258,  0.3660, -0.1397]], grad_fn=<CatBackward0>), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None),
 (tensor([[[-0.0258,  0.3660, -0.1397],
           [-0.0256,  0.2454, -0.0948],
           [-0.0135,  0.1229, -0.0555]]], grad_fn=<StackBackward0>),
  tensor([[[-0.0509,  1.0141, -0.6359],
           [-0.0707,  0.9523, -0.7242],
           [-0.0460,  0.5219, -0.4897]]], grad_fn=<StackBackward0>)))

In [ ]:
(PackedSequence(data=tensor([[ 0.0171,  0.2328, -0.0830],
         [-0.0291,  0.1918, -0.0890],
         [-0.0135,  0.1229, -0.0555],
         [-0.0063,  0.3442, -0.1289],
         [-0.0256,  0.2454, -0.0948],
         [-0.0258,  0.3660, -0.1397]], grad_fn=<CatBackward0>), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None),
 (tensor([[[-0.0258,  0.3660, -0.1397],
           [-0.0256,  0.2454, -0.0948],
           [-0.0135,  0.1229, -0.0555]]], grad_fn=<StackBackward0>),
  tensor([[[-0.0509,  1.0141, -0.6359],
           [-0.0707,  0.9523, -0.7242],
           [-0.0460,  0.5219, -0.4897]]], grad_fn=<StackBackward0>)))

In [22]:
def train(model, device, optimizer, train_dataset, val_dataset):
    accuracies = []
    for phase in ["train", "validation"]:
        if phase == "train":
            model.train(True)
        else:
            model.train(False) #or model.evel()
        correct = 0.0
        count = 0
        accuracy = None
        dataset = train_dataset if phase == "train" else val_dataset
        for sentence, tags in tqdm(dataset):
            if phase == "train":
                tag_scores, loss = model(sentence.to(device), tags.to(device))
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            else:
                with torch.no_grad():
                    tag_scores, _ = model(sentence.to(device), tags.to(device))
            correct += (tag_scores.argmax(1).to("cpu") == tags).sum()
            count += len(tags)
            accuracy = correct/count
        accuracies += [accuracy]
    return accuracies

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)
n_classes = 2
model = MyNet(tag_dim=n_classes)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=8e-3)

best_accuracy = 0
best_epoch = None
for epoch in range(1000):
    print(f"\n -- Epoch {epoch} --")
    train_accuracy, val_accuracy = train(model, device, optimizer, train_dataloader, test_dataloader)
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_epoch = epoch
    if epoch - best_epoch == 3:
        break
    
print(f"best accuracy: {best_accuracy:.2f} in epoch {best_epoch}")

device: cpu

 -- Epoch 0 --


  0%|          | 0/981 [00:00<?, ?it/s]


RuntimeError: shape '[64, -1, 30]' is invalid for input of size 12800

In [21]:
for a, b in train_dataloader:
    print(a.size(), b.size())

torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) torch.Size([64])
torch.Size([64, 200]) tor